# <center> <img src="./img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Proyecto Final: Website Activity** </center>
---

<center>

**Equipo McQueen**: Marco Albanese, Vicente Siloe

**Carrera**: Ingeniería en Sistemas Computacionales

**Fecha**: 13 de mayo del 2025

**Profesor**: Pablo Camarillo Ramirez

</center>

---
### <center> **Primavera 2025** </center>
---

### **Introducción y definición del problema**

El procesamiento de datos en aplicaciones es esencial hoy en día para una multitud de usos y propósitos que proveen beneficios tanto a usuarios como empresas. En este proyecto, creado al culminar el curso de "Procesamiento de datos masivos", se pretende crear una aplicación que integre los conocimientos adquiridos en dicho curso. A través de una *pipeline* de *Big Data*, se busca producir, consumir, analizar y procesar datos de la actividad (tráfico) observada en una aplicación web.

Los datos involucrados representan:
- Interacciones de usuario
- Páginas visitadas
- Clicks
- Dispositivos utilizados
- Etc.

Asimismo, se entrena un modelo de aprendizaje automático (*machine learning*) en base a los datos recolectados para predecir comportamientos de usuario, utilizando regresión logística. Los resultados de este modelo se presentan al usuario final a través de un *dashboard* de visualización de datos.

Para la realización de este proyecto, se utilizaron tecnologías como **Jupyter Notebooks**, **Python**, **PySpark**, **Apache Kafka** y **Power BI**. En secciones posteriores, se describe con mayor detalle la arquitectura, implementación y funcionamiento del sistema.

### **Arquitectura del sistema**

La arquitectura del sistema es la siguiente:

1. **Productores Kafka:** encargados de generar datos de actividad/tráfico web. Se utilizan dos acorde a la cantidad de integrantes en el equipo y dichos datos son publicados a dos tópicos individuales.
2. **Kafka Broker:** intermediario para conectar productores con consumidr, donde se crean los tópicos necesarios.
3. **Consumidor PySpark:** consumidor de los datos en tiempo real, guardando activamente en formato Parquet para formar un data lake.
4. **Modelo de aprendizaje máquina (Machine Learning):**
5. **Visualización de datos:** se incluye un dashboard de visualización en base a los resultados obtenidos por el modelo de *machine learning*, realizado en Power BI.

A continuación se presenta un diagrama visual de dicha arquitectura:


# <img src="./img/system_architecture.png" alt="System Architecture">

### **Justificación de las 5Vs**

- **Volumen:** En una ejecución de 60 segundos, se generan aproximadamente 50 archivos en formato Parquet. Seleccionando dichos archivos y verificando sus propiedades, obtenemos un tamaño de 370 KB. En promedio, se observa un tamaño de 7.4 KB por archivo. Teniendo esta información, podemos generar la tabla de crecimiento:

<center>

| **Periodo de tiempo** | **Datos procesados** |
|-----------------------|----------------------|
| 1 segundo             | 7.4 KB               |
| 1 minuto              | 444 KB               |
| 1 hora                | 26.64 MB             |
| 1 día                 | 639.36 MB            |
| 1 año                 | 233.1 GB             |

</center>

- **Velocidad:** La cantidad de filas procesadas por segundo en la aplicación es de 

- **Variedad:** El esquema de datos producidos por el tráfico de la página web es el siguiente:

<center>


| **Campo**           | **Tipo de dato** |
|---------------------|------------------|
| event_id            | String           |
| user_id             | String           |
| page_url            | String           |
| timestamp           | Timestamp        |
| action_type         | String           |
| browser             | String           |
| timezone            | String           |
| device_type         | String           |
| click_coordinates_x | Integer          |
| click_coordinates_y | Integer          |
| session_id          | String           |
| ip_address          | String           |

</center>


- **Veracidad:** Los datos generados para cada campo se asemejan a la realidad gracias al uso de la librería Faker. Por ejemplo, los IDs son generados en formato UUID de 128 bits, *timezone* representa una zona horaria real, *ip_address* sigue el formato de IPv4, etc. 

- **Valor:**

### **Detalles de implementación**

- Para el levantamiento del broker de Kafka como también PySpark, se utilizaron contenedores creados a través de Docker Compose, una herramienta para correr aplicaciones multi-contenedor.

- Para la creación de los productores de datos, se utilizaron dos Jupyter Notebooks localizados en la carpeta `lib`, llamados `producer_a.ipynb` y `producer_b.ipynb`. Estos mantienen parecido al ejemplo localizado en `notebooks/examples/producers/kafka_producer.ipynb`. 
  - A su vez, ambos productores importan una función auxiliar de un script de Python llamado `website_activity.py`, localizado en la carpeta `lib`. Este script utiliza la librería Faker para generar datos aleatorios.

La implementación de esta aplicación sigue una estructura similar a laboratorios vistos en clase. Notablemente, se toma como inspiración a los siguientes archivos:

- `example_structured_streaming_files_traffic_data.ipynb`
- `example_structured_streaming_files.ipynb`
- `example_structured_streaming_kafka_watermarking.ipynb`
- `example_structured_streaming_kafka.ipynb`
- `example_k_means.ipynb`

### **Resultados y evaluación**

Placeholder

### **Conclusión**

Placeholder

---

### **Código: Spark + Kafka**
---

Previo a la ejecución del siguiente código, los contenedores de Docker para el cluster de Spark y el cluster de Kafka deben estar activos.

Para ello, se puede utilizar los comandos:

`docker compose up --scale spark-worker=2 -d`

`docker compose up -d`

En los directorios `spark_cluster` y `kafka_cluster` respectivamente. Nótese que se requerirán distintos IDs de contenedores en los bloques de código pertinentes.

#### Encontrar instalación de Spark

In [3]:
import findspark
findspark.init()

#### Creación de la conexión con el cluster de Spark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Final-Project-Equipo-McQueen") \
    .master("spark://073701f82f51:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()

sc = spark.sparkContext

#### Creación de tópicos de Kafka

Los siguientes comandos deben ser ejecutados en la terminal para que los productores puedan enviar mensajes a los tópicos:

```
docker exec -it 4fcb95a9039f /opt/kafka/bin/kafka-topics.sh --create --zookeeper zookeeper:2181 --replication-factor 1 --partitions 1 --topic producer-a

docker exec -it 4fcb95a9039f /opt/kafka/bin/kafka-topics.sh --create --zookeeper zookeeper:2181 --replication-factor 1 --partitions 1 --topic producer-b
```

#### Ejecución de productores

Para inicializar a los productores, ejecute las respectivas celdas en los Jupyter Notebooks (`producer-a.ipynb`, `producer-b.ipynb`) bajo el directorio `spark_cluster/notebooks/lib/equipo_mcqueen/`

#### Creación del Kafka stream

In [5]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "4fcb95a9039f:9093") \
                .option("subscribe", "producer-a,producer-b") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



#### Transformar datos en cadenas (strings)

In [6]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

#### Creación del query (sink)

In [7]:
query = kafka_df \
                .writeStream \
                .trigger(processingTime="3 seconds") \
                .outputMode("append") \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(60)

25/05/13 06:42:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 06:42:11 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


False

#### Terminación de query y context

In [8]:
query.stop()

25/05/13 06:43:21 ERROR FileFormatWriter: Aborting job fa093cb0-03ec-4f38-98f3-562934fd3f13.
java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1048)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait0(Promise.scala:243)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:255)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:104)
	at org.apache.spark.util.ThreadUtils$.awaitReady(ThreadUtils.scala:342)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:980)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2393)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeWrite$1(FileFormatWriter.scala:241)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.writeAndCommit(FileFormatWriter.scala:271)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeWrite(

25/05/13 06:43:21 WARN TaskSetManager: Lost task 0.0 in stage 24.0 (TID 48) (172.23.0.3 executor 0): TaskKilled (Stage cancelled: Job 24 cancelled part of cancelled job group 58cb6da8-f109-4337-a611-70499033fc4d)
25/05/13 06:43:21 WARN TaskSetManager: Lost task 1.0 in stage 24.0 (TID 47) (172.23.0.4 executor 1): TaskKilled (Stage cancelled: Job 24 cancelled part of cancelled job group 58cb6da8-f109-4337-a611-70499033fc4d)


In [9]:
sc.stop()

### **Código: Machine Learning**
---

#### Encontrar instalación de Spark

In [1]:
import findspark
findspark.init()

#### Creación de la conexión con el cluster de Spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Equipo-McQueen-Logistic-Regression") \
    .master("spark://073701f82f51:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/13 06:45:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Preparación de datos

In [3]:
website_activity_df = spark.read.parquet("/home/jovyan/notebooks/data/")
website_activity_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- value_str: string (nullable = true)



#### Verificación de datos generados

In [4]:
website_activity_df.select("value_str").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value_str                                                                                                                                                                                                                                                                                                                                                                                                      

#### Creación de esquema

In [5]:
from equipo_mcqueen.spark_utils import SparkUtils

website_activity_data = [
    ("user_id", "StringType"),
    ("action_type", "StringType"),
    ("device_type", "StringType"),
    ("click_coordinates_x", "IntegerType"),
    ("click_coordinates_y", "IntegerType")
]

activity_schema = SparkUtils.generate_schema(website_activity_data)

#### Extracción de datos para modelo

In [6]:
from pyspark.sql.functions import from_json, col

df = website_activity_df.withColumn("parsed_data", from_json(col("value_str"), activity_schema))
activity_df = df.select("parsed_data.*")
activity_df.printSchema()
activity_df.show(truncate=False)

root
 |-- user_id: string (nullable = true)
 |-- action_type: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- click_coordinates_x: integer (nullable = true)
 |-- click_coordinates_y: integer (nullable = true)



+------------------------------------+-----------+-----------+-------------------+-------------------+
|user_id                             |action_type|device_type|click_coordinates_x|click_coordinates_y|
+------------------------------------+-----------+-----------+-------------------+-------------------+
|2253af06-9b48-4e33-9ec4-964363459f15|scroll     |mobile     |1434               |158                |
|2d92161b-308e-4092-8f51-c0b37fe0cdf7|click      |mobile     |640                |591                |
|5863a070-7474-42f9-b4fb-c496a1dd8a3f|view       |mobile     |509                |869                |
|b688a49a-6b80-48e8-a816-cf958d6fab32|click      |desktop    |1088               |586                |
|dec7907b-e1a8-44e3-ad2e-6dcc35eed7a7|view       |desktop    |780                |840                |
|881141f2-6686-4214-a2fe-3a63b7d9ed9d|click      |mobile     |188                |501                |
|cead986f-8156-419d-81ab-c5fe8f4b2d9e|click      |tablet     |1211       

#### Agrupación y agregación de datos

In [ ]:
from pyspark.sql.functions import first, count, avg, when

user_features = activity_df.groupBy("user_id").agg(first("device_type").alias("device_type"),
                                                   count(when(col("action_type") == "click", 1)).alias("click_count"),
                                                   count(when(col("action_type") == "scroll", 1)).alias("scroll_count"),
                                                   count(when(col("action_type") == "view", 1)).alias("view_count"),
                                                   avg("click_coordinates_x").alias("avg_click_coordinates_x"),
                                                   avg("click_coordinates_y").alias("avg_click_coordinates_y"))

user_features.show(truncate=False)

+------------------------------------+-----------+-----------+------------+----------+-----------------------+-----------------------+
|user_id                             |device_type|click_count|scroll_count|view_count|avg_click_coordinates_x|avg_click_coordinates_y|
+------------------------------------+-----------+-----------+------------+----------+-----------------------+-----------------------+
|02b1a16a-89cd-438c-8df4-14e41558f16c|mobile     |1          |0           |0         |63.0                   |843.0                  |
|03d347da-63f8-4d0c-af46-e5ade618ffcc|mobile     |1          |0           |0         |155.0                  |813.0                  |
|04830576-04b7-4c43-b590-db95ae85170c|mobile     |1          |0           |0         |1494.0                 |590.0                  |
|06366b86-fb90-4d0f-8db2-f584d4f2c9aa|mobile     |0          |0           |1         |151.0                  |92.0                   |
|06710b9b-d537-41cf-b549-44dde55e005b|desktop    |0    

#### Creación de vector

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

numerical_features = [
    "click_count",
    "scroll_count",
    "view_count", 
    "avg_click_coordinates_x",
    "avg_click_coordinates_y"
]

assembler = VectorAssembler(inputCols=numerical_features, outputCol="features")

assembled_data = assembler.transform(user_features)

#### Creación de Scaler

In [ ]:
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withStd=True,
    withMean=True
)

scaler_model = scaler.fit(assembled_data)
scaled_data = scaler_model.transform(assembled_data)

#### Creación de modelo

In [ ]:
from pyspark.ml.clustering import KMeans

k_value = 5

kmeans = KMeans(
    featuresCol="scaled_features",
    k=k_value,
    seed=42
)

#### Entrenamiento

In [ ]:
model = kmeans.fit(scaled_data)

#### Predicciones

In [ ]:
predictions = model.transform(scaled_data)

### **Visualización de datos: Power BI**
---

#### Conversión a CSV

#### Dashboard de ejemplo

Placeholder